In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import random
import lr_scheduler
import func
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report


folder_path = "F:/code/barlow/UR5"
file_pairs = func.load_file_pairs(folder_path)

AUTO = tf.data.AUTOTUNE
CROP_TO = 32
SEED = 200

PROJECT_DIM = 2048
BATCH_SIZE = 32
EPOCHS = 50

In [2]:
random_500_similar, random_500_dissimilar, remaining_similar, remaining_dissimilar = func.split_and_shuffle_pairs(file_pairs, folder_path)

def process_data(data_list):
    return [
        [func.read_jpg_files(jpg_path), func.read_and_parse_npy_file(npy_path)]
        for jpg_path, npy_path in data_list
    ]

xtrain1 = process_data(random_500_similar)
xtrain2 = process_data(random_500_dissimilar)
xtest1 = process_data(remaining_similar)
xtest2 = process_data(remaining_dissimilar)

# label 1 - similar, 0 - disimilar
ytrain1 = [1] * 500
ytrain2 = [0] * 500
ytest1 = [1] *79
ytest2 = [0] *79

y_train = ytrain1 + ytrain2
y_test = ytest1 + ytest2
x_train = xtrain1 + xtrain2
x_test = xtest1 + xtest2

In [3]:
len(x_test)

158

In [4]:
def preprocess_data(data):
    images = []
    numerical_features = []
    for image, features in data:
        # flat
        images.append(image.flatten())
        numerical_features.append(features)
    
    images = np.array(images)
    numerical_features = np.array(numerical_features)
    
    combined_features = np.hstack((images, numerical_features))
    return combined_features


X_train = preprocess_data(x_train)
X_test = preprocess_data(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Standard Scaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# SVM
clf = SVC(kernel='linear', C=1)  
clf.fit(X_train, y_train)

# fit
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.5253164556962026
Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.18      0.27        79
           1       0.51      0.87      0.65        79

    accuracy                           0.53       158
   macro avg       0.55      0.53      0.46       158
weighted avg       0.55      0.53      0.46       158



In [5]:
for i, (true_label, predicted_label) in enumerate(zip(y_test, y_pred)):
    print(f"Sample {i}: Predicted Label = {predicted_label}, True Label = {true_label}")

Sample 0: Predicted Label = 1, True Label = 1
Sample 1: Predicted Label = 1, True Label = 1
Sample 2: Predicted Label = 1, True Label = 1
Sample 3: Predicted Label = 1, True Label = 1
Sample 4: Predicted Label = 1, True Label = 1
Sample 5: Predicted Label = 1, True Label = 1
Sample 6: Predicted Label = 1, True Label = 1
Sample 7: Predicted Label = 1, True Label = 1
Sample 8: Predicted Label = 1, True Label = 1
Sample 9: Predicted Label = 1, True Label = 1
Sample 10: Predicted Label = 0, True Label = 1
Sample 11: Predicted Label = 1, True Label = 1
Sample 12: Predicted Label = 1, True Label = 1
Sample 13: Predicted Label = 1, True Label = 1
Sample 14: Predicted Label = 1, True Label = 1
Sample 15: Predicted Label = 1, True Label = 1
Sample 16: Predicted Label = 1, True Label = 1
Sample 17: Predicted Label = 1, True Label = 1
Sample 18: Predicted Label = 1, True Label = 1
Sample 19: Predicted Label = 0, True Label = 1
Sample 20: Predicted Label = 1, True Label = 1
Sample 21: Predicted La

In [6]:
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split

# # Chia tập train ban đầu thành train và validation
# X_train_new, X_val, y_train_new, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# # Tạo model SVM
# clf = SVC(kernel='linear', C=1, probability=True)

# # Các tham số
# epochs = 10
# train_accuracies = []
# val_accuracies = []
# losses = []

# for epoch in range(epochs):
#     # Huấn luyện
#     clf.fit(X_train_new, y_train_new)

#     # Dự đoán trên tập train và val
#     y_train_pred = clf.predict(X_train_new)
#     y_val_pred = clf.predict(X_val)
    
#     # Tính accuracy
#     train_accuracy = accuracy_score(y_train_new, y_train_pred)
#     val_accuracy = accuracy_score(y_val, y_val_pred)
    
#     # Sử dụng negative log loss làm "loss"
#     train_loss = -np.mean(np.log(clf.predict_proba(X_train_new)[np.arange(len(y_train_new)), y_train_new]))
#     val_loss = -np.mean(np.log(clf.predict_proba(X_val)[np.arange(len(y_val)), y_val]))

#     # Lưu lại kết quả
#     train_accuracies.append(train_accuracy)
#     val_accuracies.append(val_accuracy)
#     losses.append(train_loss)

#     # In kết quả từng epoch
#     print(f"Epoch {epoch+1}/{epochs}")
#     print(f"Train Accuracy: {train_accuracy:.4f} - Loss: {train_loss:.4f} - Val Accuracy: {val_accuracy:.4f}")

# # Dự đoán trên tập test
# y_test_pred = clf.predict(X_test)
# test_accuracy = accuracy_score(y_test, y_test_pred)
# print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# # Plot quá trình train
# plt.figure(figsize=(12, 6))

# plt.subplot(1, 2, 1)
# plt.plot(range(1, epochs+1), train_accuracies, label='Train Accuracy')
# plt.plot(range(1, epochs+1), val_accuracies, label='Val Accuracy', linestyle='--')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.title('Accuracy over Epochs')
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.plot(range(1, epochs+1), losses, label='Train Loss', color='red')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Loss over Epochs')
# plt.legend()

# plt.tight_layout()
# plt.show()
